In [13]:
# 1. 기본 라이브러리 및 설정
import os
import asyncio
import json
import logging
import re
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, Union
from dataclasses import dataclass, field
from enum import Enum

# 데이터베이스 관련
import pymysql
from pymysql.cursors import DictCursor
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Row
from sqlalchemy.exc import OperationalError

# AI/LLM 관련
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 설정 관리
from dotenv import load_dotenv

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# 환경변수 로드
load_dotenv("C:/Users/Administrator/Desktop/skala10/SKoro-AI/.env", override=True)

# ====================================
# 2. 예외 클래스 정의
# ====================================

class Module11Error(Exception):
    """모듈 11 전용 예외"""
    pass

class DatabaseError(Module11Error):
    """데이터베이스 관련 예외"""
    pass

# ====================================
# 3. 데이터베이스 설정
# ====================================

# DB 연결 설정
db_type = os.getenv("DB_TYPE")  # mariadb
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# MariaDB/MySQL의 경우 mysql+pymysql 드라이버 사용
if db_type.lower() in ["mariadb", "mysql"]:
    DATABASE_URL = f"mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
else:
    DATABASE_URL = f"{db_type}://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

# DB 엔진 생성
engine = create_engine(DATABASE_URL, pool_pre_ping=True)

# LLM 클라이언트 설정
llm_client = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [ ]:
# ====================================
# 4. 유틸리티 함수
# ====================================

def row_to_dict(row: Row) -> Dict:
    """SQLAlchemy Row 객체를 딕셔너리로 변환"""
    if row is None:
        return {}
    return row._asdict()



# ====================================
# 5. 데이터 클래스
# ====================================

@dataclass
class Module11AgentState:
    # 필수 키값들
    team_id: int
    period_id: int
    team_evaluation_id: int
    is_final: bool
    
    # 분석 과정에서 도출된 핵심 인사이트만 (가벼운 데이터)
    key_risks: Optional[List[str]] = None
    collaboration_bias_level: Optional[str] = None  # "high", "medium", "low"
    performance_trend: Optional[str] = None
    
    # 최종 JSON 결과
    ai_risk_result: Optional[Dict[str, Any]] = None
    ai_plan_result: Optional[Dict[str, Any]] = None
    overall_comment_result: Optional[str] = None  # TEXT 형태

# ====================================
# 6. DB 래퍼 클래스
# ====================================

class SQLAlchemyDBWrapper:
    """수정된 DB 래퍼 클래스"""
    
    def __init__(self, engine):
        self.engine = engine
    
    def fetch_one(self, query, params=None):
        """단일 행 조회"""
        with self.engine.connect() as conn:
            result = conn.execute(text(query), params or {})
            row = result.fetchone()
            return row._asdict() if row else None
    
    def fetch_all(self, query, params=None):
        """전체 행 조회"""
        with self.engine.connect() as conn:
            result = conn.execute(text(query), params or {})
            rows = result.fetchall()
            return [row._asdict() for row in rows] if rows else []
    
    def execute_update(self, query, params=None):
        """업데이트 실행 (수정된 버전)"""
        with self.engine.begin() as conn:  # 🔥 자동 트랜잭션 관리
            result = conn.execute(text(query), params or {})
            affected_rows = result.rowcount
            
            # 명시적 검증
            if affected_rows == 0:
                logger.warning(f"업데이트된 행이 없음: query={query[:100]}...")
            else:
                logger.info(f"업데이트 완료: {affected_rows}행 영향")
            
            return affected_rows

# DB 래퍼 클래스 초기화
db_wrapper = SQLAlchemyDBWrapper(engine)

In [ ]:
# ====================================
# 7. 메인 Module11 클래스
# ====================================

class Module11TeamRiskManagementAgent:
    def __init__(self, db_connection):
        self.db = db_connection
    
    def _parse_json_field(self, json_str: Optional[str]) -> Optional[Dict[str, Any]]:
        """JSON 필드 안전 파싱 (클래스 메서드)"""
        if not json_str:
            return None
        
        try:
            return json.loads(json_str)
        except (json.JSONDecodeError, TypeError) as e:
            logger.warning(f"JSON 파싱 실패: {str(e)[:100]}...")
            return None
    
    def _extract_json_from_llm_response(self, text: str) -> str:
        """개선된 LLM 응답에서 JSON 블록 추출"""
        
        # 1. 마크다운 코드 블록에서 JSON 추출 (가장 일반적)
        json_block_pattern = r"```(?:json)?\s*(.*?)\s*```"
        match = re.search(json_block_pattern, text, re.DOTALL | re.IGNORECASE)
        if match:
            json_content = match.group(1).strip()
            if self._is_valid_json_string(json_content):
                return json_content
        
        # 2. 중괄호로 둘러싸인 JSON 객체 찾기 (중첩 고려)
        json_objects = self._extract_nested_json_objects(text)
        for json_obj in json_objects:
            if self._is_valid_json_string(json_obj):
                return json_obj
        
        # 3. 전체 텍스트가 JSON인지 확인
        stripped_text = text.strip()
        if stripped_text.startswith('{') and stripped_text.endswith('}'):
            if self._is_valid_json_string(stripped_text):
                return stripped_text
        
        # 4. 최후의 수단: 첫 번째 { 부터 마지막 } 까지
        first_brace = text.find('{')
        last_brace = text.rfind('}')
        if first_brace != -1 and last_brace != -1 and first_brace < last_brace:
            potential_json = text[first_brace:last_brace + 1]
            if self._is_valid_json_string(potential_json):
                return potential_json
        
        logger.warning(f"JSON 추출 실패, 원본 텍스트 반환: {text[:200]}...")
        return text.strip()
        
    async def execute(self, team_id: int, period_id: int, team_evaluation_id: int) -> Module11AgentState:
        """모듈 11 메인 실행 함수"""
        logger.info(f"Module11 시작: team_id={team_id}, period_id={period_id}")
        
        try:
            # 1. State 초기화
            state = Module11AgentState(
                team_id=team_id,
                period_id=period_id,
                team_evaluation_id=team_evaluation_id,
                is_final=self._check_is_final(period_id)
            )
            
            # 2. 데이터 수집
            data = self._collect_all_data_sequential(state)
            
            # 3. 리스크 분석
            state.ai_risk_result = await self._analyze_parallel(state, data)
            
            # 4. 결과 생성 
            state = await self._generate_outputs_parallel(state, data)
            
            # 5. 저장
            self._save_results(state)
            
            logger.info(f"Module11 완료: team_id={team_id}")
            return state
            
        except Exception as e:
            logger.error(f"Module11 실행 중 오류: {str(e)}")
            raise Module11Error(f"모듈 11 실행 실패: {str(e)}")
        
    def _extract_nested_json_objects(self, text: str) -> List[str]:
        """중첩된 JSON 객체들을 추출"""
        json_objects = []
        brace_count = 0
        start_pos = -1
        
        for i, char in enumerate(text):
            if char == '{':
                if brace_count == 0:
                    start_pos = i
                brace_count += 1
            elif char == '}':
                brace_count -= 1
                if brace_count == 0 and start_pos != -1:
                    json_candidate = text[start_pos:i + 1]
                    json_objects.append(json_candidate)
                    start_pos = -1
        
        # 길이순으로 정렬 (더 완전한 JSON이 뒤에 오도록)
        return sorted(json_objects, key=len, reverse=True)
    
    def _is_valid_json_string(self, text: str) -> bool:
        """JSON 문자열 유효성 검사"""
        try:
            json.loads(text)
            return True
        except (json.JSONDecodeError, TypeError):
            return False
    
    def _check_is_final(self, period_id: int) -> bool:
        """기간이 연말인지 확인"""
        try:
            query = "SELECT is_final FROM periods WHERE period_id = :period_id"
            result = self.db.fetch_one(query, {'period_id': period_id})
            return bool(result['is_final']) if result else False
        except Exception as e:
            logger.error(f"연말 여부 확인 실패: {str(e)}")
            raise DatabaseError(f"기간 정보 조회 실패: {str(e)}")
    
    def _collect_all_data_sequential(self, state: Module11AgentState) -> Dict[str, Any]:
        """순차적 데이터 수집 (실제 테이블 구조에 맞춤)"""
        logger.info(f"데이터 수집 시작: team_evaluation_id={state.team_evaluation_id}")
        
        data = {}
        
        try:
            # 1. 기본 정보
            data['period_info'] = self._get_period_info(state.period_id)
            data['team_info'] = self._get_team_info(state.team_id)
            data['team_members'] = self._get_team_members(state.team_id)
            
            # 2. 성과 데이터
            year = data['period_info']['year']
            data['team_kpis'] = self._get_team_kpis(state.team_id, year)
            data['team_performance'] = self._get_team_performance(state.team_id, state.period_id)
            
            # 3. 협업 분석 (JSON 파싱)
            collaboration_data = self._get_collaboration_data(state.team_evaluation_id)
            data['collaboration_matrix'] = self._parse_json_field(collaboration_data.get('ai_collaboration_matrix'))
            data['team_coaching'] = collaboration_data.get('ai_team_coaching')  # TEXT 필드
            
            # ai_team_comparison은 분기에만 필요
            if not state.is_final:
                data['team_comparison'] = collaboration_data.get('ai_team_comparison')  # TEXT 필드
            
            # 4. 개인 리스크
            data['individual_risks'] = self._get_individual_risks(state.team_evaluation_id, state.is_final)
            
            # 5. 분기별/연말별 추가 데이터
            if not state.is_final:
                # 분기: 전분기 데이터
                data['previous_quarter'] = self._get_previous_quarter_data(state.team_id, data['period_info'])
            else:
                # 연말: temp_evaluations 데이터
                data['temp_evaluations'] = self._get_temp_evaluations(state.team_id)
            
            logger.info(f"데이터 수집 완료: {len(data)} 개 데이터셋")
            return data
            
        except Exception as e:
            logger.error(f"데이터 수집 실패: {str(e)}")
            raise DatabaseError(f"데이터 수집 중 오류 발생: {str(e)}")
    
    # ====================================
    # 데이터 수집 메서드들 (실제 테이블 구조에 맞춤)
    # ====================================
    
    def _get_period_info(self, period_id: int) -> Dict[str, Any]:
        """기간 정보 조회"""
        query = """
        SELECT period_id, year, period_name, order_in_year, is_final,
               start_date, end_date
        FROM periods 
        WHERE period_id = :period_id
        """
        
        result = self.db.fetch_one(query, {'period_id': period_id})
        if not result:
            raise DatabaseError(f"기간 정보를 찾을 수 없음: period_id={period_id}")
        
        return dict(result)

    def _get_team_info(self, team_id: int) -> Dict[str, Any]:
        """팀 기본 정보 조회"""
        query = """
        SELECT t.team_id, t.team_name, t.team_description,
               h.headquarter_id, h.headquarter_name, 
               p.part_id, p.part_name
        FROM teams t
        JOIN headquarters h ON t.headquarter_id = h.headquarter_id
        JOIN parts p ON h.part_id = p.part_id
        WHERE t.team_id = :team_id
        """
        
        result = self.db.fetch_one(query, {'team_id': team_id})
        if not result:
            raise DatabaseError(f"팀 정보를 찾을 수 없음: team_id={team_id}")
        
        return dict(result)

    def _get_team_members(self, team_id: int) -> List[Dict[str, Any]]:
        """팀원 목록 조회"""
        query = """
        SELECT emp_no, emp_name, email, cl, position, role, salary
        FROM employees 
        WHERE team_id = :team_id
        ORDER BY cl DESC, emp_no
        """
        
        results = self.db.fetch_all(query, {'team_id': team_id})
        if not results:
            logger.warning(f"팀원이 없음: team_id={team_id}")
        
        return [dict(row) for row in results]

    def _get_team_kpis(self, team_id: int, year: int) -> List[Dict[str, Any]]:
        """팀 KPI 조회"""
        query = """
        SELECT team_kpi_id, kpi_name, kpi_description, weight, 
               ai_kpi_progress_rate, ai_kpi_analysis_comment
        FROM team_kpis 
        WHERE team_id = :team_id AND year = :year
        ORDER BY weight DESC, team_kpi_id
        """
        
        results = self.db.fetch_all(query, {'team_id': team_id, 'year': year})
        if not results:
            logger.warning(f"팀 KPI가 없음: team_id={team_id}, year={year}")
        
        return [dict(row) for row in results]

    def _get_team_performance(self, team_id: int, period_id: int) -> Dict[str, Any]:
        """팀 성과 지표 조회 (실제 테이블 구조에 맞춤)"""
        query = """
        SELECT average_achievement_rate, relative_performance, year_over_year_growth,
               ai_team_overall_analysis_comment
        FROM team_evaluations 
        WHERE team_id = :team_id AND period_id = :period_id
        """
        
        result = self.db.fetch_one(query, {'team_id': team_id, 'period_id': period_id})
        if not result:
            logger.warning(f"팀 성과 데이터 없음: team_id={team_id}, period_id={period_id}")
            return {}
        
        return dict(result)

    def _get_collaboration_data(self, team_evaluation_id: int) -> Dict[str, Any]:
        """협업 분석 데이터 조회"""
        query = """
        SELECT ai_collaboration_matrix, ai_team_comparison, ai_team_coaching
        FROM team_evaluations 
        WHERE team_evaluation_id = :team_evaluation_id
        """
        
        result = self.db.fetch_one(query, {'team_evaluation_id': team_evaluation_id})
        if not result:
            logger.warning(f"협업 분석 데이터 없음: team_evaluation_id={team_evaluation_id}")
            return {}
        
        return dict(result)

    def _get_individual_risks(self, team_evaluation_id: int, is_final: bool) -> List[Dict[str, Any]]:
        """개인별 리스크 분석 결과 조회 (실제 테이블 구조에 맞춤)"""
        if is_final:
            # 연말: final_evaluation_reports
            query = """
            SELECT emp_no, ai_growth_coaching, score, contribution_rate,
                   ai_annual_performance_summary_comment, cl_reason
            FROM final_evaluation_reports 
            WHERE team_evaluation_id = :team_evaluation_id
            ORDER BY score DESC
            """
        else:
            # 분기: feedback_reports
            query = """
            SELECT emp_no, ai_growth_coaching, contribution_rate,
                   ai_overall_contribution_summary_comment, attitude
            FROM feedback_reports 
            WHERE team_evaluation_id = :team_evaluation_id
            ORDER BY contribution_rate DESC
            """
        
        results = self.db.fetch_all(query, {'team_evaluation_id': team_evaluation_id})
        
        # ai_growth_coaching JSON 파싱
        parsed_results = []
        for row in results:
            row_dict = dict(row)
            row_dict['ai_growth_coaching'] = self._parse_json_field(row.get('ai_growth_coaching'))
            parsed_results.append(row_dict)
        
        if not parsed_results:
            logger.warning(f"개인 평가 데이터 없음: team_evaluation_id={team_evaluation_id}")
        
        return parsed_results

    def _get_previous_quarter_data(self, team_id: int, period_info: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        """전분기 데이터 조회 (분기용)"""
        current_year = period_info['year']
        current_order = period_info['order_in_year']
        
        # 이전 분기 계산
        if current_order > 1:
            # 같은 해 이전 분기
            prev_year = current_year
            prev_order = current_order - 1
        else:
            # 전년도 4분기
            prev_year = current_year - 1
            prev_order = 4
        
        query = """
        SELECT te.average_achievement_rate, te.relative_performance, 
               te.ai_risk, te.overall_comment,
               p.period_name
        FROM team_evaluations te
        JOIN periods p ON te.period_id = p.period_id
        WHERE te.team_id = :team_id AND p.year = :year AND p.order_in_year = :order_in_year
        LIMIT 1
        """
        
        result = self.db.fetch_one(query, {
            'team_id': team_id, 
            'year': prev_year, 
            'order_in_year': prev_order
        })
        
        if result:
            result_dict = dict(result)
            result_dict['ai_risk'] = self._parse_json_field(result.get('ai_risk'))
            # overall_comment는 TEXT이므로 JSON 파싱하지 않음
            logger.info(f"전분기 데이터 조회 완료: {prev_year}년 {prev_order}분기")
            return result_dict
        
        logger.warning(f"전분기 데이터 없음: team_id={team_id}, {prev_year}년 {prev_order}분기")
        return None

    def _get_temp_evaluations(self, team_id: int) -> List[Dict[str, Any]]:
        """중간평가 데이터 조회 (연말용) - DB에서 직접 조회"""
        logger.info(f"DB에서 temp_evaluations 조회: team_id={team_id}")
        
        query = """
        SELECT te.temp_evaluation_id, te.ai_reason, te.score, te.raw_score,
               te.manager_score, te.comment, te.reason, te.status, te.emp_no
        FROM temp_evaluations te
        WHERE te.emp_no IN (
            SELECT emp_no FROM employees WHERE team_id = :team_id
        )
        ORDER BY te.score DESC
        """
        
        results = self.db.fetch_all(query, {'team_id': team_id})
        
        if not results:
            logger.warning(f"중간평가 데이터 없음: team_id={team_id}")
            return []
        
        logger.info(f"중간평가 데이터 조회 완료: {len(results)}건")
        return [dict(row) for row in results]

    # ====================================
    # 8. 리스크 분석 메서드들 (수정된 구조)
    # ====================================

    async def _analyze_parallel(self, state: Module11AgentState, data: Dict[str, Any]) -> Dict[str, Any]:
        """LLM 기반 병렬 리스크 분석 (수정된 구조)"""
        logger.info("LLM 기반 리스크 분석 시작 (병렬)")
        
        try:
            # 독립적인 LLM 분석들을 병렬 처리
            tasks = [
                self._analyze_collaboration_risks_with_llm_async(data.get('collaboration_matrix'), data.get('team_members', [])),
                self._analyze_individual_risk_patterns_with_llm_async(data.get('individual_risks', []), data.get('team_members', [])),
                self._analyze_performance_trends_with_llm_async(data.get('team_performance', {}), data.get('team_kpis', []))
            ]
            
            risk_analyses = await asyncio.gather(*tasks)
            
            # LLM으로 최종 통합 분석 (새로운 구조로)
            integrated_analysis = await self._integrate_risk_analysis_with_llm_async(
                collaboration_risks=risk_analyses[0],
                individual_patterns=risk_analyses[1], 
                performance_trends=risk_analyses[2],
                state=state,
                data=data
            )
            
            logger.info("LLM 기반 리스크 분석 완료")
            return integrated_analysis
            
        except Exception as e:
            logger.error(f"LLM 리스크 분석 실패: {str(e)}")
            raise Module11Error(f"LLM 리스크 분석 중 오류: {str(e)}")

    async def _integrate_risk_analysis_with_llm_async(self, collaboration_risks: Dict, individual_patterns: Dict, 
                                                    performance_trends: Dict, state: 'Module11AgentState', data: Dict) -> Dict[str, Any]:
        """개선된 LLM 기반 리스크 분석 통합 및 최종 JSON 생성"""
        
        # 팀 기본 정보
        team_info = data.get('team_info', {})
        period_info = data.get('period_info', {})
        team_members = data.get('team_members', [])
        team_performance = data.get('team_performance', {})
        
        prompt = f"""
당신은 팀 운영 및 리스크 관리 전문가입니다. 다음 세부 분석 결과들을 종합하여 team_evaluations.ai_risk에 저장될 구조화된 JSON을 생성해주세요.

## 팀 기본 정보:
- 팀명: {team_info.get('team_name', '')}
- 평가기간: {period_info.get('period_name', '')} ({'연말' if state.is_final else '분기'} 평가)
- 팀원 수: {len(team_members)}명
- 팀 성과: 달성률 {team_performance.get('average_achievement_rate', 0)}%, 상대성과 {team_performance.get('relative_performance', 0)}%

## 협업 리스크 분석 결과:
{json.dumps(collaboration_risks, ensure_ascii=False, indent=2)}

## 개인별 리스크 패턴 분석 결과:
{json.dumps(individual_patterns, ensure_ascii=False, indent=2)}

## 성과 트렌드 분석 결과:
{json.dumps(performance_trends, ensure_ascii=False, indent=2)}

위 분석 결과들을 종합하여 다음과 같은 정확한 구조의 JSON을 생성해주세요:

```json
{{
  "risk_analysis": {{
    "major_risks": [
      {{
        "risk_name": "실제 데이터에 기반한 구체적 리스크명",
        "severity": "high/medium/low",
        "description": "분석 결과를 종합한 구체적이고 실용적인 리스크 설명",
        "causes": [
          "데이터에서 확인된 구체적 발생 원인 1",
          "데이터에서 확인된 구체적 발생 원인 2"
        ],
        "impacts": [
          {{
            "impact_scope": "individual/team/organization",
            "timeline": "immediate/short_term/long_term",
            "description": "실제 예상되는 구체적 영향 설명"
          }}
        ],
        "strategies": [
          {{
            "description": "데이터 분석 결과에 기반한 구체적이고 실행 가능한 운영 개선 전략"
          }}
        ]
      }}
    ]
  }}
}}
```

## 중요 요구사항:
1. 실제 분석 결과를 기반으로 한 구체적이고 실용적인 내용 작성
2. major_risks는 심각도 순으로 정렬하여 최대 5개까지
3. 각 리스크는 실제 데이터 근거와 함께 설명
4. causes, impacts, strategies는 추상적이지 않고 구체적으로 작성
5. 팀 실정에 맞는 실행 가능한 전략 제시
6. JSON 형식을 정확히 준수
"""

        try:
            response = await asyncio.to_thread(llm_client.invoke, prompt)
            result_json = self._extract_json_from_llm_response(response.content)
            final_result = json.loads(result_json)
            
            # 추가 검증 및 후처리
            if not self._validate_risk_json_structure(final_result):
                logger.warning("리스크 JSON 구조 검증 실패 - 폴백 사용")
                return self._create_fallback_risk_analysis(state)
            
            # State에 핵심 정보 저장
            major_risks = final_result.get('risk_analysis', {}).get('major_risks', [])
            state.key_risks = [risk['risk_name'] for risk in major_risks]
            state.collaboration_bias_level = collaboration_risks.get('bias_level', 'medium')
            
            logger.info(f"✅ 고품질 리스크 분석 완료: {len(major_risks)}개 리스크 식별")
            return final_result
            
        except Exception as e:
            logger.error(f"최종 리스크 분석 통합 실패: {str(e)}")
            return self._create_fallback_risk_analysis(state)
        
    def _validate_risk_json_structure(self, result: Dict) -> bool:
        """리스크 JSON 구조 검증"""
        try:
            risk_analysis = result.get('risk_analysis', {})
            major_risks = risk_analysis.get('major_risks', [])
            
            if not isinstance(major_risks, list) or len(major_risks) == 0:
                return False
            
            # 첫 번째 리스크의 필수 필드 검증
            first_risk = major_risks[0]
            required_fields = ['risk_name', 'severity', 'description', 'causes', 'impacts', 'strategies']
            
            for field in required_fields:
                if field not in first_risk:
                    return False
            
            return True
            
        except Exception:
            return False

    def _create_fallback_risk_analysis(self, state: 'Module11AgentState') -> Dict[str, Any]:
        """개선된 폴백 리스크 분석"""
        
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M")
        
        return {
            "risk_analysis": {
                "major_risks": [
                    {
                        "risk_name": "시스템 분석 한계로 인한 리스크 식별 제약",
                        "severity": "medium",
                        "description": f"LLM 분석 프로세스에서 기술적 한계가 발생하여 정확한 리스크 식별이 제한됨 (발생시간: {current_time})",
                        "causes": [
                            "자동화된 분석 시스템의 기술적 한계",
                            "복잡한 조직 데이터 처리 과정의 예외 상황"
                        ],
                        "impacts": [
                            {
                                "impact_scope": "team",
                                "timeline": "immediate",
                                "description": "정확한 리스크 대응 계획 수립의 어려움 및 의사결정 지연 가능성"
                            }
                        ],
                        "strategies": [
                            {
                                "description": "팀장 주도의 수동 리스크 검토 프로세스 즉시 시행 및 주요 리스크 요소 직접 식별"
                            }
                        ]
                    }
                ]
            }
        }

    # 기존 개별 분석 메서드들
    async def _analyze_collaboration_risks_with_llm_async(self, collaboration_matrix: Optional[Dict], team_members: List[Dict]) -> Dict[str, Any]:
        """개선된 LLM 기반 협업 리스크 분석"""
        
        if not collaboration_matrix:
            return {
                'risks': [],
                'collaboration_insights': ['협업 데이터 부족으로 분석 불가'],
                'bias_level': 'unknown'
            }
        
        # 팀원 정보 구조화
        member_info = []
        for member in team_members:
            member_info.append({
                'emp_no': member.get('emp_no'),
                'name': member.get('emp_name'),
                'position': member.get('position'),
                'cl': member.get('cl')
            })
        
        prompt = f"""
당신은 조직 협업 분석 전문가입니다. 다음 협업 매트릭스 데이터를 분석하여 구체적인 리스크를 식별해주세요.

## 팀원 정보:
{json.dumps(member_info, ensure_ascii=False, indent=2)}

## 협업 매트릭스 데이터:
{json.dumps(collaboration_matrix, ensure_ascii=False, indent=2)}

협업 리스크를 다음 JSON 구조로 분석해주세요:

```json
{{
  "risks": [
    {{
      "risk_name": "구체적 협업 리스크명",
      "severity": "high/medium/low",
      "description": "리스크에 대한 구체적 설명 (데이터 근거 포함)",
      "evidence": [
        "협업 매트릭스에서 발견된 구체적 증거 1",
        "협업 매트릭스에서 발견된 구체적 증거 2"
      ],
      "affected_members": ["emp_no1", "emp_no2"]
    }}
  ],
  "collaboration_insights": [
    "협업 패턴에서 발견된 주요 인사이트 1",
    "협업 패턴에서 발견된 주요 인사이트 2"
  ],
  "bias_level": "high/medium/low"
}}
```

## 분석 기준:
1. 협업 매트릭스의 실제 수치를 기반으로 분석
2. 팀원간 협업 불균형, 소외된 구성원, 과도한 의존성 등을 식별
3. 구체적인 데이터 근거와 함께 리스크 설명
4. 실제 영향받을 팀원들을 명시
"""

        try:
            response = await asyncio.to_thread(llm_client.invoke, prompt)
            result_json = self._extract_json_from_llm_response(response.content)
            return json.loads(result_json)
        except Exception as e:
            logger.error(f"협업 리스크 LLM 분석 실패: {str(e)}")
            return {
                'risks': [{
                    'risk_name': '협업 데이터 분석 오류',
                    'severity': 'medium',
                    'description': f'LLM 분석 중 오류 발생: {str(e)[:100]}',
                    'evidence': ['시스템 분석 한계'],
                    'affected_members': []
                }],
                'collaboration_insights': ['시스템 분석 한계로 인한 제한적 분석'],
                'bias_level': 'unknown'
            }

    async def _analyze_individual_risk_patterns_with_llm_async(self, individual_risks: List[Dict], team_members: List[Dict]) -> Dict[str, Any]:
        """개선된 LLM 기반 개인별 리스크 패턴 분석"""
        
        if not individual_risks:
            return {
                'risks': [],
                'performance_patterns': ['개인 평가 데이터 부족으로 분석 불가']
            }
        
        # 개인별 성과 데이터 구조화
        performance_data = []
        for risk in individual_risks:
            performance_data.append({
                'emp_no': risk.get('emp_no'),
                'score': risk.get('score'),
                'contribution_rate': risk.get('contribution_rate'),
                'attitude': risk.get('attitude'),
                'growth_coaching': risk.get('ai_growth_coaching'),
                'summary_comment': risk.get('ai_overall_contribution_summary_comment') or risk.get('ai_annual_performance_summary_comment')
            })
        
        prompt = f"""
당신은 인사 성과 분석 전문가입니다. 다음 개인별 성과 데이터를 분석하여 팀 차원의 리스크를 식별해주세요.

## 팀원 기본 정보:
{json.dumps([{'emp_no': m.get('emp_no'), 'name': m.get('emp_name'), 'position': m.get('position'), 'cl': m.get('cl')} for m in team_members], ensure_ascii=False, indent=2)}

## 개인별 성과 데이터:
{json.dumps(performance_data, ensure_ascii=False, indent=2)}

개인 성과 패턴을 분석하여 다음 JSON 구조로 팀 리스크를 도출해주세요:

```json
{{
  "risks": [
    {{
      "risk_name": "개인 성과 기반 팀 리스크명",
      "severity": "high/medium/low", 
      "description": "성과 데이터를 근거로 한 구체적 리스크 설명",
      "affected_members": ["emp_no1", "emp_no2"],
      "evidence": [
        "성과 데이터에서 발견된 구체적 증거 1",
        "성과 데이터에서 발견된 구체적 증거 2"
      ]
    }}
  ],
  "performance_patterns": [
    "팀 성과 패턴에서 발견된 주요 인사이트 1",
    "팀 성과 패턴에서 발견된 주요 인사이트 2"
  ]
}}
```

## 분석 포인트:
1. 성과 편차가 큰 구성원들의 팀 영향도
2. 저성과자의 팀 사기 및 분위기 영향
3. 고성과자의 번아웃 또는 이직 위험성
4. 성장 코칭 필요성이 높은 구성원들의 패턴
5. 실제 수치를 근거로 구체적 분석
"""

        try:
            response = await asyncio.to_thread(llm_client.invoke, prompt)
            result_json = self._extract_json_from_llm_response(response.content)
            return json.loads(result_json)
        except Exception as e:
            logger.error(f"개인 리스크 LLM 분석 실패: {str(e)}")
            return {
                'risks': [{
                    'risk_name': '개인 성과 분석 한계',
                    'severity': 'medium',
                    'description': f'개인 성과 데이터 분석 중 오류: {str(e)[:100]}',
                    'affected_members': [],
                    'evidence': ['시스템 분석 한계']
                }],
                'performance_patterns': ['시스템 한계로 인한 제한적 분석']
            }

    async def _analyze_performance_trends_with_llm_async(self, team_performance: Dict, team_kpis: List[Dict]) -> Dict[str, Any]:
        """개선된 LLM 기반 성과 트렌드 분석"""
        
        if not team_performance:
            return {
                'risks': [],
                'trends': ['성과 데이터 부족으로 분석 불가']
            }
        
        # KPI 데이터 구조화
        kpi_summary = []
        for kpi in team_kpis:
            kpi_summary.append({
                'kpi_name': kpi.get('kpi_name'),
                'weight': kpi.get('weight'),
                'progress_rate': kpi.get('ai_kpi_progress_rate'),
                'analysis_comment': kpi.get('ai_kpi_analysis_comment')
            })
        
        prompt = f"""
당신은 팀 성과 분석 전문가입니다. 다음 성과 데이터를 종합 분석하여 팀의 성과 관련 리스크를 식별해주세요.

## 팀 전체 성과:
{json.dumps(team_performance, ensure_ascii=False, indent=2)}

## 팀 KPI 현황:
{json.dumps(kpi_summary, ensure_ascii=False, indent=2)}

성과 트렌드를 분석하여 다음 JSON 구조로 리스크를 도출해주세요:

```json
{{
  "risks": [
    {{
      "risk_name": "성과 관련 구체적 리스크명",
      "severity": "high/medium/low",
      "description": "성과 데이터를 근거로 한 구체적 리스크 설명",
      "affected_kpis": ["kpi_name1", "kpi_name2"],
      "evidence": [
        "성과 데이터에서 발견된 구체적 수치적 근거 1",
        "성과 데이터에서 발견된 구체적 수치적 근거 2"
      ]
    }}
  ],
  "performance_trends": [
    "성과 트렌드에서 발견된 주요 패턴 1",
    "성과 트렌드에서 발견된 주요 패턴 2"
  ]
}}
```

## 분석 기준:
1. 평균 달성률, 상대 성과, 전년 대비 성장률의 실제 수치 분석
2. KPI별 진행률과 가중치를 고려한 위험도 평가
3. 성과 트렌드의 지속가능성 및 개선 가능성 판단
4. 구체적 수치를 근거로 한 명확한 리스크 식별
"""

        try:
            response = await asyncio.to_thread(llm_client.invoke, prompt)
            result_json = self._extract_json_from_llm_response(response.content)
            return json.loads(result_json)
        except Exception as e:
            logger.error(f"성과 트렌드 LLM 분석 실패: {str(e)}")
            return {
                'risks': [{
                    'risk_name': '성과 데이터 분석 한계',
                    'severity': 'medium',
                    'description': f'성과 분석 중 오류 발생: {str(e)[:100]}',
                    'affected_kpis': [],
                    'evidence': ['시스템 분석 한계']
                }],
                'performance_trends': ['시스템 한계로 인한 제한적 분석']
            }


    # ====================================
    # 9. 결과 생성 메서드들 (수정된 구조)
    # ====================================

    async def _generate_outputs_parallel(self, state: Module11AgentState, data: Dict[str, Any]) -> Module11AgentState:
        """LLM 기반 병렬 결과 생성"""
        logger.info("최종 결과 생성 시작 (병렬)")
        
        try:
            if state.is_final:
                # 연말: ai_plan과 overall_comment 병렬 생성
                tasks = [
                    self._generate_annual_plan_with_llm_async(state, data),
                    self._generate_overall_comment_with_llm_async(state, data)
                ]
                
                ai_plan, overall_comment = await asyncio.gather(*tasks)
                state.ai_plan_result = ai_plan
                state.overall_comment_result = overall_comment
                
            else:
                # 분기: overall_comment만
                state.overall_comment_result = await self._generate_overall_comment_with_llm_async(state, data)
            
            logger.info("최종 결과 생성 완료")
            return state
            
        except Exception as e:
            logger.error(f"최종 결과 생성 실패: {str(e)}")
            raise Module11Error(f"최종 결과 생성 중 오류: {str(e)}")

    async def _generate_annual_plan_with_llm_async(self, state: 'Module11AgentState', data: Dict[str, Any]) -> Dict[str, Any]:
        """개선된 LLM 기반 연말 계획 생성"""
        
        team_info = data.get('team_info', {})
        period_info = data.get('period_info', {})
        team_members = data.get('team_members', [])
        temp_evaluations = data.get('temp_evaluations', [])
        individual_risks = data.get('individual_risks', [])
        
        # 팀원별 중간평가 결과 구조화
        member_evaluations = {}
        for temp_eval in temp_evaluations:
            emp_no = temp_eval.get('emp_no')
            if emp_no:
                member_evaluations[emp_no] = {
                    'score': temp_eval.get('score'),
                    'manager_score': temp_eval.get('manager_score'),
                    'reason': temp_eval.get('reason'),
                    'status': temp_eval.get('status')
                }
        
        prompt = f"""
당신은 팀 운영 전략 수립 전문가입니다. 다음 종합 데이터를 바탕으로 차년도({period_info.get('year', 2024) + 1}년) 팀 운영 계획을 수립해주세요.

## 팀 기본 정보:
{json.dumps(team_info, ensure_ascii=False, indent=2)}

## 팀원 현황:
{json.dumps([{'emp_no': m.get('emp_no'), 'name': m.get('emp_name'), 'position': m.get('position'), 'cl': m.get('cl'), 'salary': m.get('salary')} for m in team_members], ensure_ascii=False, indent=2)}

## 중간평가 결과:
{json.dumps(member_evaluations, ensure_ascii=False, indent=2)}

## 최종 평가 결과:
{json.dumps(individual_risks, ensure_ascii=False, indent=2)}

## 식별된 리스크:
{json.dumps(state.ai_risk_result, ensure_ascii=False, indent=2)}

## 협업 분석 결과:
{json.dumps(data.get('collaboration_matrix'), ensure_ascii=False, indent=2)}

차년도 운영 계획을 다음과 같은 정확한 JSON 구조로 작성해주세요:

```json
{{
  "annual_plans": [
    {{
      "personnel_strategies": [
        {{
          "target": "구체적 대상자명 또는 포지션",
          "action": "실제 실행 가능한 구체적 방안 (교육, 승진, 채용, 역할 변경 등)"
        }}
      ],
      "collaboration_improvements": [
        {{
          "current_issue": "현재 확인된 구체적 협업 문제점",
          "improvement": "문제 해결을 위한 구체적 개선 방안",
          "expected_benefit": "개선으로 인한 구체적 기대효과",
          "target": "측정 가능한 구체적 목표 지표"
        }}
      ]
    }}
  ]
}}
```

## 작성 가이드라인:
1. **personnel_strategies**: 
   - 실제 팀원들의 성과와 평가 결과를 기반으로 구체적 전략 수립
   - 고성과자 유지, 저성과자 개선, 신규 채용 필요성 등을 실데이터 기반으로 판단
   - 실제 실행 가능한 액션 아이템으로 작성

2. **collaboration_improvements**: 
   - 협업 매트릭스와 리스크 분석 결과를 기반으로 실제 문제점 식별
   - 구체적이고 실행 가능한 개선 방안 제시
   - 측정 가능한 목표 지표 설정

3. **전체적으로**: 
   - 추상적이지 않고 구체적인 내용
   - 실제 데이터와 분석 결과에 기반
   - 차년도에 실제로 실행할 수 있는 계획
"""
        
        try:
            response = await asyncio.to_thread(llm_client.invoke, prompt)
            result_json = self._extract_json_from_llm_response(response.content)
            final_result = json.loads(result_json)
            
            # 구조 검증
            if not self._validate_plan_json_structure(final_result):
                logger.warning("연말 계획 JSON 구조 검증 실패 - 폴백 사용")
                return self._create_fallback_annual_plan(period_info)
            
            logger.info("✅ 고품질 연말 계획 생성 완료")
            return final_result
            
        except Exception as e:
            logger.error(f"연말 계획 LLM 생성 실패: {str(e)}")
            return self._create_fallback_annual_plan(period_info)
        
    def _validate_plan_json_structure(self, result: Dict) -> bool:
        """연말 계획 JSON 구조 검증"""
        try:
            annual_plans = result.get('annual_plans', [])
            if not isinstance(annual_plans, list) or len(annual_plans) == 0:
                return False
            
            plan = annual_plans[0]
            if 'personnel_strategies' not in plan or 'collaboration_improvements' not in plan:
                return False
            
            return True
            
        except Exception:
            return False

    def _create_fallback_annual_plan(self, period_info: Dict) -> Dict[str, Any]:
        """개선된 폴백 연말 계획"""
        
        next_year = period_info.get('year', 2024) + 1
        
        return {
            "annual_plans": [
                {
                    "personnel_strategies": [
                        {
                            "target": "팀 전체 구성원",
                            "action": f"{next_year}년 상반기 중 팀장 주도의 개별 면담을 통한 맞춤형 성장 계획 수립 및 실행"
                        },
                        {
                            "target": "시스템 분석 프로세스",
                            "action": "자동화 분석 시스템의 한계 보완을 위한 수동 검토 체계 구축 및 정기적 모니터링 실시"
                        }
                    ],
                    "collaboration_improvements": [
                        {
                            "current_issue": "자동화된 협업 분석의 기술적 한계",
                            "improvement": "팀 내 정기적 협업 효과성 점검 미팅 및 피드백 수집 체계 구축",
                            "expected_benefit": "실제 협업 이슈의 신속한 발견 및 해결을 통한 팀 효율성 향상",
                            "target": "월 1회 협업 점검 미팅 실시 및 분기별 협업 만족도 80% 이상 달성"
                        }
                    ]
                }
            ]
        }

    async def _generate_overall_comment_with_llm_async(self, state: 'Module11AgentState', data: Dict[str, Any]) -> str:
        """개선된 LLM 기반 총평 생성"""
        
        if state.is_final:
            return await self._generate_annual_overall_comment_text_improved(state, data)
        else:
            return await self._generate_quarterly_overall_comment_text_improved(state, data)

    async def _generate_annual_overall_comment_text_improved(self, state: 'Module11AgentState', data: Dict[str, Any]) -> str:
        """개선된 연말 총평 생성"""
        
        team_info = data.get('team_info', {})
        period_info = data.get('period_info', {})
        team_performance = data.get('team_performance', {})
        team_kpis = data.get('team_kpis', [])
        temp_evaluations = data.get('temp_evaluations', [])
        
        # 성과 지표 요약
        performance_summary = {
            'achievement_rate': team_performance.get('average_achievement_rate', 0),
            'relative_performance': team_performance.get('relative_performance', 0),
            'year_over_year_growth': team_performance.get('year_over_year_growth', 0),
            'kpi_count': len(team_kpis),
            'team_size': len(data.get('team_members', [])),
            'temp_eval_avg': sum(eval.get('score', 0) for eval in temp_evaluations) / len(temp_evaluations) if temp_evaluations else 0
        }
        
        prompt = f"""
당신은 경영진에게 보고하는 팀 운영 전략 전문가입니다. 다음 종합 데이터를 바탕으로 연말 총평을 작성해주세요.

## 팀 기본 정보:
- 팀명: {team_info.get('team_name')} 
- 소속: {team_info.get('part_name')} > {team_info.get('headquarter_name')}
- 팀원 수: {performance_summary['team_size']}명

## 연간 성과 요약:
- 평균 달성률: {performance_summary['achievement_rate']}%
- 상대적 성과: {performance_summary['relative_performance']}%  
- 전년 대비 성장률: {performance_summary['year_over_year_growth']}%
- KPI 개수: {performance_summary['kpi_count']}개
- 중간평가 평균: {performance_summary['temp_eval_avg']:.1f}점

## 상세 KPI 현황:
{json.dumps([{'name': kpi.get('kpi_name'), 'weight': kpi.get('weight'), 'progress': kpi.get('ai_kpi_progress_rate')} for kpi in team_kpis], ensure_ascii=False, indent=2)}

## 식별된 주요 리스크:
{json.dumps(state.key_risks, ensure_ascii=False)}

## 차년도 계획 요약:
{json.dumps(state.ai_plan_result, ensure_ascii=False, indent=2)}

다음 구조로 연말 총평을 작성해주세요:

**[팀 성과 방향]**
연간 성과 달성도와 성장 궤적을 구체적 수치와 함께 평가하고, 팀의 전략적 방향성과 성과 창출 능력을 분석해주세요. (3-4문장)

**[구조적 인식]**  
팀의 조직적 강점과 구조적 도전과제를 리스크 분석 결과와 연계하여 설명하고, 지속가능한 성장을 위한 핵심 요소를 식별해주세요. (3-4문장)

**[향후 운영 전략]**
차년도 계획과 연계하여 전략적 우선순위와 성공을 위한 핵심 실행 과제를 제시하고, 구체적인 성과 목표와 실행 방안을 제시해주세요. (3-4문장)

## 작성 요구사항:
1. 실제 수치와 데이터를 근거로 구체적으로 작성
2. 추상적 표현보다는 실용적이고 실행 가능한 내용 중심
3. 경영진이 의사결정에 활용할 수 있는 명확한 인사이트 제공
4. 각 섹션은 독립적이면서도 전체적으로 일관된 스토리 구성
"""

        try:
            response = await asyncio.to_thread(llm_client.invoke, prompt)
            result = response.content.strip()
            
            # 내용 품질 검증
            if len(result) < 300:
                logger.warning("연말 총평이 너무 짧음 - 폴백 사용")
                return self._create_fallback_annual_comment_text()
            
            logger.info(f"✅ 고품질 연말 총평 생성 완료: {len(result)}자")
            return result
            
        except Exception as e:
            logger.error(f"연말 총평 LLM 생성 실패: {str(e)}")
            return self._create_fallback_annual_comment_text()

    async def _generate_quarterly_overall_comment_text_improved(self, state: 'Module11AgentState', data: Dict[str, Any]) -> str:
        """개선된 분기 총평 생성"""
        
        team_info = data.get('team_info', {})
        period_info = data.get('period_info', {})
        team_performance = data.get('team_performance', {})
        previous_quarter = data.get('previous_quarter', {})
        team_comparison = data.get('team_comparison', '')
        
        # 전분기 대비 변화 계산
        current_achievement = team_performance.get('average_achievement_rate', 0)
        prev_achievement = previous_quarter.get('average_achievement_rate', 0) if previous_quarter else 0
        achievement_change = current_achievement - prev_achievement
        
        prompt = f"""
당신은 분기별 성과 분석 전문가입니다. 다음 데이터를 바탕으로 분기 총평을 작성해주세요.

## 팀 기본 정보:
- 팀명: {team_info.get('team_name')}
- 평가 기간: {period_info.get('period_name')}

## 현재 분기 성과:
- 평균 달성률: {current_achievement}%
- 상대적 성과: {team_performance.get('relative_performance', 0)}%

## 전분기 성과 (비교):
- 전분기 달성률: {prev_achievement}%
- 달성률 변화: {achievement_change:+.1f}%p

## 유사팀 비교 분석:
{team_comparison}

## 식별된 주요 리스크:
{json.dumps(state.key_risks, ensure_ascii=False)}

다음 구조로 분기 총평을 작성해주세요:

**[전분기 대비 변화]**
전분기 대비 주요 변화사항과 성과 트렌드를 구체적 수치와 함께 분석해주세요. (2-3문장)

**[유사조직 대비 현황]**
상대적 위치와 벤치마킹 인사이트를 제시하고, 개선 또는 유지해야 할 포인트를 명확히 해주세요. (2-3문장)

**[종합 평가]**
핵심 인사이트와 다음 분기까지 즉시 집중해야 할 영역을 실행 가능한 액션과 함께 제시해주세요. (2-3문장)

## 작성 요구사항:
1. 구체적 수치와 변화량을 반드시 포함
2. 실행 가능한 개선 방향 제시
3. 다음 분기 성과 향상을 위한 명확한 가이드 제공
"""

        try:
            response = await asyncio.to_thread(llm_client.invoke, prompt)
            result = response.content.strip()
            
            # 내용 품질 검증
            if len(result) < 200:
                logger.warning("분기 총평이 너무 짧음 - 폴백 사용")
                return self._create_fallback_quarterly_comment_text()
            
            logger.info(f"✅ 고품질 분기 총평 생성 완료: {len(result)}자")
            return result
            
        except Exception as e:
            logger.error(f"분기 총평 LLM 생성 실패: {str(e)}")
            return self._create_fallback_quarterly_comment_text()
        

    def _create_fallback_annual_comment_text(self) -> str:
        """개선된 폴백 연말 총평"""
        return """**[팀 성과 방향]**
시스템 분석의 기술적 한계로 인해 정확한 연간 성과 분석이 제한되었으나, 팀의 지속적인 성장과 발전을 위해서는 체계적인 성과 관리와 목표 달성 모니터링이 필요합니다. 자동화된 분석 시스템을 보완하는 수동 검토 프로세스를 통해 더욱 정확한 성과 평가 체계를 구축해야 합니다.

**[구조적 인식]**
현재 조직의 데이터 기반 의사결정 시스템이 완전하지 않음을 인식하고, 이를 보완할 수 있는 다층적 검토 체계가 필요합니다. 팀장의 직접적인 관찰과 판단을 통한 정성적 평가와 시스템 분석을 결합하여 보다 균형잡힌 조직 운영이 가능할 것입니다.

**[향후 운영 전략]**  
차년도에는 시스템 의존도를 줄이고 팀장 주도의 능동적 팀 관리 체계를 강화하는 것이 우선순위입니다. 정기적인 개별 면담, 팀 내 소통 활성화, 그리고 구체적이고 측정 가능한 목표 설정을 통해 팀 성과를 지속적으로 향상시켜 나가야 합니다."""

    def _create_fallback_quarterly_comment_text(self) -> str:
        """개선된 폴백 분기 총평"""
        return """**[전분기 대비 변화]**
시스템 분석 한계로 인해 정확한 전분기 대비 변화를 수치적으로 파악하기 어려우나, 지속적인 성과 관리를 위해서는 팀장의 직접적인 관찰과 평가가 필요합니다.

**[유사조직 대비 현황]**
비교 데이터의 한계로 유사조직과의 정확한 벤치마킹이 제한적이지만, 팀 고유의 강점을 활용한 차별화된 성과 창출에 집중해야 합니다.

**[종합 평가]**
다음 분기에는 시스템에 의존하지 않는 독립적인 성과 모니터링 체계를 구축하고, 팀원들과의 직접적인 소통을 통해 실질적인 성과 개선 방안을 도출해야 합니다."""
    # ====================================
    # 10. 저장 메서드들
    # ====================================

    def _save_results(self, state) -> None:
        """분석 결과를 DB에 저장 (수정된 버전)"""
        logger.info(f"분석 결과 저장 시작: team_evaluation_id={state.team_evaluation_id}")
        
        try:
            # 1. 저장할 데이터 준비
            save_data = self._prepare_save_data(state)
            
            if not save_data:
                logger.warning("저장할 데이터가 없습니다.")
                return
            
            # 2. 저장 전 검증 (존재하는 레코드인지 확인)
            if not self._verify_team_evaluation_exists(state.team_evaluation_id):
                raise DatabaseError(f"team_evaluation_id {state.team_evaluation_id}가 존재하지 않습니다.")
            
            # 3. 실제 업데이트
            affected_rows = self._update_team_evaluations(state.team_evaluation_id, save_data)
            
            if affected_rows == 0:
                raise DatabaseError(f"업데이트된 행이 없음: team_evaluation_id={state.team_evaluation_id}")
            
            # 4. 저장 후 검증
            self._verify_save_success(state.team_evaluation_id, save_data)
            
            logger.info(f"✅ 분석 결과 저장 완료: team_evaluation_id={state.team_evaluation_id}")
            
        except Exception as e:
            logger.error(f"❌ 분석 결과 저장 실패: {str(e)}")
            raise DatabaseError(f"DB 저장 중 오류 발생: {str(e)}")

    def _prepare_save_data(self, state) -> dict:
        """저장용 데이터 준비"""
        save_data = {}
        
        # ai_risk는 JSON으로 저장
        if state.ai_risk_result:
            save_data['ai_risk'] = json.dumps(state.ai_risk_result, ensure_ascii=False)
            logger.info(f"ai_risk 데이터 준비 완료: {len(save_data['ai_risk'])}자")
        
        # ai_plan은 JSON으로 저장 (연말만)
        if state.is_final and state.ai_plan_result:
            save_data['ai_plan'] = json.dumps(state.ai_plan_result, ensure_ascii=False)
            logger.info(f"ai_plan 데이터 준비 완료: {len(save_data['ai_plan'])}자")
        
        # overall_comment는 텍스트로 저장
        if state.overall_comment_result:
            save_data['overall_comment'] = state.overall_comment_result
            logger.info(f"overall_comment 데이터 준비 완료: {len(save_data['overall_comment'])}자")
        
        logger.info(f"총 {len(save_data)}개 필드 준비 완료: {list(save_data.keys())}")
        return save_data
    
    def _verify_team_evaluation_exists(self, team_evaluation_id: int) -> bool:
        """team_evaluation 레코드 존재 여부 확인"""
        query = "SELECT COUNT(*) as cnt FROM team_evaluations WHERE team_evaluation_id = :team_evaluation_id"
        result = self.db.fetch_one(query, {'team_evaluation_id': team_evaluation_id})
        
        exists = result['cnt'] > 0 if result else False
        logger.info(f"team_evaluation_id {team_evaluation_id} 존재 여부: {exists}")
        return exists

    def _update_team_evaluations(self, team_evaluation_id: int, save_data: dict) -> int:
        """team_evaluations 테이블 업데이트 (수정된 버전)"""
        
        # UPDATE 쿼리 동적 생성
        set_clauses = []
        params = {'team_evaluation_id': team_evaluation_id}
        
        for column, value in save_data.items():
            set_clauses.append(f"{column} = :{column}")
            params[column] = value
        
        query = f"""
        UPDATE team_evaluations 
        SET {', '.join(set_clauses)}
        WHERE team_evaluation_id = :team_evaluation_id
        """
        
        logger.info(f"실행할 쿼리: {query}")
        logger.info(f"파라미터 키: {list(params.keys())}")
        
        # 실제 업데이트 실행
        affected_rows = self.db.execute_update(query, params)
        
        return affected_rows
    
    def _verify_save_success(self, team_evaluation_id: int, save_data: dict) -> None:
        """저장 성공 여부 검증"""
        logger.info("저장 결과 검증 시작...")
        
        # 저장된 데이터 다시 조회
        columns = list(save_data.keys())
        query = f"""
        SELECT {', '.join(columns)}
        FROM team_evaluations 
        WHERE team_evaluation_id = :team_evaluation_id
        """
        
        result = self.db.fetch_one(query, {'team_evaluation_id': team_evaluation_id})
        
        if not result:
            raise DatabaseError("저장 검증 실패: 데이터를 다시 조회할 수 없음")
        
        # 각 필드별 검증
        for column in columns:
            saved_value = result.get(column)
            if not saved_value:
                logger.warning(f"⚠️ {column} 필드가 비어있음")
            else:
                logger.info(f"✅ {column} 저장 확인: {len(str(saved_value))}자")
        
        logger.info("저장 결과 검증 완료")
    


# ====================================
# 11. 실행 함수
# ====================================

async def run_module_11(team_id: int, period_id: int, team_evaluation_id: int) -> Module11AgentState:
    """Module 11 실행 함수"""
    
    try:
        # Agent 초기화
        agent = Module11TeamRiskManagementAgent(db_wrapper)
        
        # 실행
        result = await agent.execute(team_id, period_id, team_evaluation_id)
        
        return result
        
    except Exception as e:
        logger.error(f"Module 11 실행 실패: {str(e)}")
        raise


-----
test

In [40]:
# ====================================
# Module 11 개선 버전 테스트 코드
# ====================================

import asyncio
import json
import logging
from datetime import datetime
from typing import Dict, Any

# ====================================
# 1. 테스트 파라미터 (실제 데이터)
# ====================================

# 실제 존재하는 데이터
TEST_TEAM_ID = 900001  # AI핵심기술팀
TEST_PERIOD_ID = 800007  # 2025년 4분기 (연말)
TEST_TEAM_EVALUATION_ID = 700001  # 해당 팀 평가

# ====================================
# 2. 개선 전후 비교 테스트
# ====================================

async def test_improved_module11():
    """개선된 Module 11 전체 실행 테스트"""
    
    print("🚀 개선된 Module 11 테스트 시작")
    print("=" * 80)
    print(f"📋 테스트 파라미터:")
    print(f"   팀 ID: {TEST_TEAM_ID} (AI핵심기술팀)")
    print(f"   기간 ID: {TEST_PERIOD_ID} (2025년 4분기 - 연말)")
    print(f"   평가 ID: {TEST_TEAM_EVALUATION_ID}")
    print("=" * 80)
    
    try:
        # 실행 전 DB 상태 확인
        print("\n🔍 실행 전 DB 상태 확인...")
        before_data = check_db_before_test(TEST_TEAM_EVALUATION_ID)
        
        # 개선된 Module 11 실행
        print("\n⚡ 개선된 Module 11 실행 중...")
        start_time = datetime.now()
        
        result = await run_module_11(TEST_TEAM_ID, TEST_PERIOD_ID, TEST_TEAM_EVALUATION_ID)
        
        end_time = datetime.now()
        execution_time = (end_time - start_time).total_seconds()
        
        print(f"✅ 실행 완료! (소요시간: {execution_time:.1f}초)")
        
        # 실행 후 DB 상태 확인
        print("\n🔍 실행 후 DB 상태 확인...")
        after_data = check_db_after_test(TEST_TEAM_EVALUATION_ID)
        
        # 결과 분석
        print("\n📊 결과 분석...")
        analyze_test_results(result, before_data, after_data, execution_time)
        
        # JSON 구조 검증
        print("\n🔬 JSON 구조 검증...")
        validate_json_structures(after_data)
        
        # 내용 품질 검증
        print("\n📝 내용 품질 검증...")
        validate_content_quality(after_data)
        
        # 최종 결과 요약
        print("\n📋 최종 테스트 결과 요약...")
        summarize_test_results(result, before_data, after_data)
        
        return result
        
    except Exception as e:
        print(f"❌ 테스트 실패: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# ====================================
# 3. DB 상태 확인 함수들
# ====================================

def check_db_before_test(team_evaluation_id: int) -> Dict[str, Any]:
    """테스트 실행 전 DB 상태 확인"""
    
    query = """
    SELECT ai_risk, ai_plan, overall_comment,
           ai_collaboration_matrix, ai_team_coaching, ai_team_comparison,
           updated_at
    FROM team_evaluations 
    WHERE team_evaluation_id = :team_evaluation_id
    """
    
    result = db_wrapper.fetch_one(query, {'team_evaluation_id': team_evaluation_id})
    
    if result:
        print("📊 실행 전 상태:")
        print(f"   ai_risk: {'있음' if result['ai_risk'] else '없음'} ({len(str(result['ai_risk'])) if result['ai_risk'] else 0}자)")
        print(f"   ai_plan: {'있음' if result['ai_plan'] else '없음'} ({len(str(result['ai_plan'])) if result['ai_plan'] else 0}자)")
        print(f"   overall_comment: {'있음' if result['overall_comment'] else '없음'} ({len(str(result['overall_comment'])) if result['overall_comment'] else 0}자)")
        print(f"   마지막 업데이트: {result.get('updated_at', 'Unknown')}")
        
        return dict(result)
    else:
        print("❌ 테스트 대상 데이터를 찾을 수 없음")
        return {}

def check_db_after_test(team_evaluation_id: int) -> Dict[str, Any]:
    """테스트 실행 후 DB 상태 확인"""
    
    query = """
    SELECT ai_risk, ai_plan, overall_comment,
           ai_collaboration_matrix, ai_team_coaching, ai_team_comparison,
           updated_at
    FROM team_evaluations 
    WHERE team_evaluation_id = :team_evaluation_id
    """
    
    result = db_wrapper.fetch_one(query, {'team_evaluation_id': team_evaluation_id})
    
    if result:
        print("📊 실행 후 상태:")
        print(f"   ai_risk: {'생성됨' if result['ai_risk'] else '생성 실패'} ({len(str(result['ai_risk'])) if result['ai_risk'] else 0}자)")
        print(f"   ai_plan: {'생성됨' if result['ai_plan'] else '생성 실패'} ({len(str(result['ai_plan'])) if result['ai_plan'] else 0}자)")
        print(f"   overall_comment: {'생성됨' if result['overall_comment'] else '생성 실패'} ({len(str(result['overall_comment'])) if result['overall_comment'] else 0}자)")
        print(f"   마지막 업데이트: {result.get('updated_at', 'Unknown')}")
        
        return dict(result)
    else:
        print("❌ 테스트 결과 데이터를 찾을 수 없음")
        return {}

# ====================================
# 4. 결과 분석 함수들
# ====================================

def analyze_test_results(result: Any, before_data: Dict, after_data: Dict, execution_time: float):
    """테스트 결과 종합 분석"""
    
    print("🎯 테스트 결과 분석:")
    print(f"   실행 시간: {execution_time:.1f}초")
    print(f"   연말 평가 여부: {result.is_final if result else 'Unknown'}")
    print(f"   식별된 리스크 수: {len(result.key_risks) if result and result.key_risks else 0}개")
    print(f"   협업 편향 수준: {result.collaboration_bias_level if result else 'Unknown'}")
    
    # 데이터 생성 여부 확인
    module11_fields = ['ai_risk', 'ai_plan', 'overall_comment']
    
    for field in module11_fields:
        before_len = len(str(before_data.get(field, ''))) if before_data.get(field) else 0
        after_len = len(str(after_data.get(field, ''))) if after_data.get(field) else 0
        
        if after_len > before_len:
            print(f"   ✅ {field}: 성공 ({before_len} → {after_len}자, +{after_len - before_len})")
        elif after_len == before_len and after_len > 0:
            print(f"   ⚠️ {field}: 변경 없음 ({after_len}자)")
        else:
            print(f"   ❌ {field}: 생성 실패 ({before_len} → {after_len}자)")

def validate_json_structures(after_data: Dict[str, Any]):
    """JSON 구조 검증"""
    
    print("🔬 JSON 구조 검증:")
    
    # ai_risk JSON 구조 검증
    if after_data.get('ai_risk'):
        try:
            risk_data = json.loads(after_data['ai_risk'])
            
            # 필수 구조 확인
            if 'risk_analysis' in risk_data:
                major_risks = risk_data['risk_analysis'].get('major_risks', [])
                print(f"   ✅ ai_risk: 올바른 JSON 구조 (주요 리스크 {len(major_risks)}개)")
                
                # 첫 번째 리스크 구조 확인
                if major_risks:
                    first_risk = major_risks[0]
                    required_fields = ['risk_name', 'severity', 'description', 'causes', 'impacts', 'strategies']
                    missing_fields = [f for f in required_fields if f not in first_risk]
                    
                    if not missing_fields:
                        print(f"      ✅ 리스크 필드 완전성: 모든 필수 필드 존재")
                    else:
                        print(f"      ⚠️ 리스크 필드 누락: {missing_fields}")
                else:
                    print(f"      ⚠️ 주요 리스크가 비어있음")
            else:
                print(f"   ❌ ai_risk: 잘못된 JSON 구조 (risk_analysis 키 없음)")
                
        except json.JSONDecodeError as e:
            print(f"   ❌ ai_risk: JSON 파싱 실패 - {str(e)}")
    else:
        print(f"   ❌ ai_risk: 데이터 없음")
    
    # ai_plan JSON 구조 검증
    if after_data.get('ai_plan'):
        try:
            plan_data = json.loads(after_data['ai_plan'])
            
            if 'annual_plans' in plan_data:
                annual_plans = plan_data['annual_plans']
                print(f"   ✅ ai_plan: 올바른 JSON 구조 (연간 계획 {len(annual_plans)}개)")
                
                # 계획 구조 확인
                if annual_plans:
                    first_plan = annual_plans[0]
                    if 'personnel_strategies' in first_plan and 'collaboration_improvements' in first_plan:
                        personnel_count = len(first_plan['personnel_strategies'])
                        collaboration_count = len(first_plan['collaboration_improvements'])
                        print(f"      ✅ 계획 구성: 인사전략 {personnel_count}개, 협업개선 {collaboration_count}개")
                    else:
                        print(f"      ⚠️ 계획 구조 불완전")
                else:
                    print(f"      ⚠️ 연간 계획이 비어있음")
            else:
                print(f"   ❌ ai_plan: 잘못된 JSON 구조 (annual_plans 키 없음)")
                
        except json.JSONDecodeError as e:
            print(f"   ❌ ai_plan: JSON 파싱 실패 - {str(e)}")
    else:
        print(f"   ❌ ai_plan: 데이터 없음")

def validate_content_quality(after_data: Dict[str, Any]):
    """내용 품질 검증"""
    
    print("📝 내용 품질 검증:")
    
    # ai_risk 내용 품질 확인
    if after_data.get('ai_risk'):
        risk_content = after_data['ai_risk']
        
        # 기본 품질 지표
        quality_indicators = {
            '구체적 수치 포함': any(char.isdigit() for char in risk_content),
            '한국어 분석': '팀' in risk_content or '성과' in risk_content,
            '실행 가능한 전략': '전략' in risk_content or '개선' in risk_content,
            '충분한 내용량': len(risk_content) > 500
        }
        
        print("   📋 ai_risk 품질:")
        for indicator, passed in quality_indicators.items():
            status = "✅" if passed else "❌"
            print(f"      {status} {indicator}")
    
    # overall_comment 내용 품질 확인
    if after_data.get('overall_comment'):
        comment_content = after_data['overall_comment']
        
        quality_indicators = {
            '구조화된 형식': '[' in comment_content and ']' in comment_content,
            '구체적 수치 포함': any(char.isdigit() for char in comment_content),
            '실행 가능한 내용': '전략' in comment_content or '계획' in comment_content,
            '충분한 분량': len(comment_content) > 300
        }
        
        print("   💬 overall_comment 품질:")
        for indicator, passed in quality_indicators.items():
            status = "✅" if passed else "❌"
            print(f"      {status} {indicator}")

def summarize_test_results(result: Any, before_data: Dict, after_data: Dict):
    """최종 테스트 결과 요약"""
    
    print("=" * 80)
    print("📋 최종 테스트 결과 요약")
    print("=" * 80)
    
    # 전체 성공률 계산
    total_fields = 3  # ai_risk, ai_plan, overall_comment
    successful_fields = 0
    
    if after_data.get('ai_risk'):
        successful_fields += 1
    if after_data.get('ai_plan'):
        successful_fields += 1
    if after_data.get('overall_comment'):
        successful_fields += 1
    
    success_rate = (successful_fields / total_fields) * 100
    
    print(f"🎯 전체 성공률: {success_rate:.1f}% ({successful_fields}/{total_fields})")
    
    # 개선 효과 분석
    improvements = []
    
    # 내용 길이 개선
    for field in ['ai_risk', 'ai_plan', 'overall_comment']:
        before_len = len(str(before_data.get(field, ''))) if before_data.get(field) else 0
        after_len = len(str(after_data.get(field, ''))) if after_data.get(field) else 0
        
        if after_len > before_len:
            improvement = after_len - before_len
            improvements.append(f"{field}: +{improvement}자")
    
    if improvements:
        print(f"📈 개선 효과: {', '.join(improvements)}")
    
    # JSON 구조 개선 여부
    json_improvements = []
    
    if after_data.get('ai_risk'):
        try:
            risk_data = json.loads(after_data['ai_risk'])
            if 'risk_analysis' in risk_data:
                json_improvements.append("ai_risk 구조화")
        except:
            pass
    
    if after_data.get('ai_plan'):
        try:
            plan_data = json.loads(after_data['ai_plan'])
            if 'annual_plans' in plan_data:
                json_improvements.append("ai_plan 구조화")
        except:
            pass
    
    if json_improvements:
        print(f"🔧 구조 개선: {', '.join(json_improvements)}")
    
    # 권장 사항
    print("\n💡 권장 사항:")
    if success_rate < 100:
        print("   - 실패한 필드에 대한 오류 로그 확인 필요")
        print("   - LLM 프롬프트 추가 최적화 고려")
    
    if success_rate >= 80:
        print("   ✅ 전반적으로 우수한 성능")
        print("   - 프로덕션 환경 적용 고려 가능")
    elif success_rate >= 60:
        print("   ⚠️ 부분적 성공 - 추가 개선 필요")
        print("   - 실패 원인 분석 후 재테스트 권장")
    else:
        print("   ❌ 성능 개선 필요")
        print("   - 기본 설정 및 환경 재점검 필요")
    
    print("=" * 80)

# ====================================
# 5. 개별 구성요소 테스트
# ====================================

async def test_individual_components():
    """개별 구성요소별 테스트"""
    
    print("\n🧪 개별 구성요소 테스트")
    print("-" * 50)
    
    # Agent 초기화
    agent = Module11TeamRiskManagementAgent(db_wrapper)
    
    # 테스트용 State 생성
    from dataclasses import dataclass
    
    @dataclass
    class TestState:
        team_id: int = TEST_TEAM_ID
        period_id: int = TEST_PERIOD_ID
        team_evaluation_id: int = TEST_TEAM_EVALUATION_ID
        is_final: bool = True
        key_risks: list = None
        collaboration_bias_level: str = None
        ai_risk_result: dict = None
    
    state = TestState()
    
    # 실제 데이터 수집
    print("📊 실제 데이터 수집 중...")
    try:
        data = agent._collect_all_data_sequential(state)
        print(f"   ✅ 데이터 수집 완료: {len(data)}개 데이터셋")
        
        # 각 구성요소별 테스트
        print("\n🔍 협업 리스크 분석 테스트...")
        collaboration_result = await agent._analyze_collaboration_risks_with_llm_async(
            data.get('collaboration_matrix'), 
            data.get('team_members', [])
        )
        print(f"   결과: {len(collaboration_result.get('risks', []))}개 협업 리스크 식별")
        
        print("\n🔍 개인 리스크 패턴 분석 테스트...")
        individual_result = await agent._analyze_individual_risk_patterns_with_llm_async(
            data.get('individual_risks', []), 
            data.get('team_members', [])
        )
        print(f"   결과: {len(individual_result.get('risks', []))}개 개인 리스크 식별")
        
        print("\n🔍 성과 트렌드 분석 테스트...")
        performance_result = await agent._analyze_performance_trends_with_llm_async(
            data.get('team_performance', {}), 
            data.get('team_kpis', [])
        )
        print(f"   결과: {len(performance_result.get('risks', []))}개 성과 리스크 식별")
        
        return {
            'collaboration': collaboration_result,
            'individual': individual_result,
            'performance': performance_result
        }
        
    except Exception as e:
        print(f"   ❌ 개별 구성요소 테스트 실패: {str(e)}")
        return None

# ====================================
# 6. 메인 테스트 실행
# ====================================

async def run_all_tests():
    """모든 테스트 실행"""
    
    print("🚀 Module 11 개선 버전 종합 테스트")
    print("=" * 80)
    
    # 1. 전체 통합 테스트
    print("\n1️⃣ 전체 통합 테스트")
    integration_result = await test_improved_module11()
    
    # 2. 개별 구성요소 테스트
    print("\n2️⃣ 개별 구성요소 테스트")
    component_results = await test_individual_components()
    
    # 3. 성능 비교 테스트 (옵션)
    print("\n3️⃣ 성능 비교 테스트 (기존 vs 개선)")
    # 여러 번 실행하여 평균 성능 측정 가능
    
    print("\n🎉 모든 테스트 완료!")
    
    return {
        'integration': integration_result,
        'components': component_results
    }

# ====================================
# 7. 실행 방법
# ====================================


test_results = await run_all_tests()

# # 또는 개별 테스트 실행
# result = await test_improved_module11()

# # 또는 구성요소별 테스트
# components = await test_individual_components()


# # Jupyter 환경에서 직접 실행
# print("🧪 테스트 시작...")
# test_results = await run_all_tests()

2025-06-24 19:46:36,987 - __main__ - INFO - Module11 시작: team_id=900001, period_id=800007
2025-06-24 19:46:37,007 - __main__ - INFO - 데이터 수집 시작: team_evaluation_id=700001


🚀 Module 11 개선 버전 종합 테스트

1️⃣ 전체 통합 테스트
🚀 개선된 Module 11 테스트 시작
📋 테스트 파라미터:
   팀 ID: 900001 (AI핵심기술팀)
   기간 ID: 800007 (2025년 4분기 - 연말)
   평가 ID: 700001

🔍 실행 전 DB 상태 확인...
📊 실행 전 상태:
   ai_risk: 없음 (0자)
   ai_plan: 없음 (0자)
   overall_comment: 없음 (0자)
   마지막 업데이트: None

⚡ 개선된 Module 11 실행 중...


2025-06-24 19:46:37,136 - __main__ - WARNING - 개인 평가 데이터 없음: team_evaluation_id=700001
2025-06-24 19:46:37,138 - __main__ - INFO - DB에서 temp_evaluations 조회: team_id=900001
2025-06-24 19:46:37,196 - __main__ - INFO - 중간평가 데이터 조회 완료: 5건
2025-06-24 19:46:37,197 - __main__ - INFO - 데이터 수집 완료: 9 개 데이터셋
2025-06-24 19:46:37,198 - __main__ - INFO - LLM 기반 리스크 분석 시작 (병렬)
2025-06-24 19:46:45,414 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-24 19:46:47,214 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-24 19:47:06,714 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-24 19:47:06,721 - __main__ - INFO - ✅ 고품질 리스크 분석 완료: 5개 리스크 식별
2025-06-24 19:47:06,721 - __main__ - INFO - LLM 기반 리스크 분석 완료
2025-06-24 19:47:06,722 - __main__ - INFO - 최종 결과 생성 시작 (병렬)
2025-06-24 19:47:13,190 - httpx - INFO - HTTP Request: POST https://api.openai.

✅ 실행 완료! (소요시간: 40.2초)

🔍 실행 후 DB 상태 확인...
📊 실행 후 상태:
   ai_risk: 생성됨 (2620자)
   ai_plan: 생성됨 (1182자)
   overall_comment: 생성됨 (696자)
   마지막 업데이트: None

📊 결과 분석...
🎯 테스트 결과 분석:
   실행 시간: 40.2초
   연말 평가 여부: True
   식별된 리스크 수: 5개
   협업 편향 수준: medium
   ✅ ai_risk: 성공 (0 → 2620자, +2620)
   ✅ ai_plan: 성공 (0 → 1182자, +1182)
   ✅ overall_comment: 성공 (0 → 696자, +696)

🔬 JSON 구조 검증...
🔬 JSON 구조 검증:
   ✅ ai_risk: 올바른 JSON 구조 (주요 리스크 5개)
      ✅ 리스크 필드 완전성: 모든 필수 필드 존재
   ✅ ai_plan: 올바른 JSON 구조 (연간 계획 1개)
      ✅ 계획 구성: 인사전략 3개, 협업개선 3개

📝 내용 품질 검증...
📝 내용 품질 검증:
   📋 ai_risk 품질:
      ✅ 구체적 수치 포함
      ✅ 한국어 분석
      ✅ 실행 가능한 전략
      ✅ 충분한 내용량
   💬 overall_comment 품질:
      ✅ 구조화된 형식
      ✅ 구체적 수치 포함
      ✅ 실행 가능한 내용
      ✅ 충분한 분량

📋 최종 테스트 결과 요약...
📋 최종 테스트 결과 요약
🎯 전체 성공률: 100.0% (3/3)
📈 개선 효과: ai_risk: +2620자, ai_plan: +1182자, overall_comment: +696자
🔧 구조 개선: ai_risk 구조화, ai_plan 구조화

💡 권장 사항:
   ✅ 전반적으로 우수한 성능
   - 프로덕션 환경 적용 고려 가능

2️⃣ 개별 구성요소 테스트

🧪 개별 구성요소 테스트
---------------------------

2025-06-24 19:47:17,367 - __main__ - INFO - 데이터 수집 완료: 9 개 데이터셋


   ✅ 데이터 수집 완료: 9개 데이터셋

🔍 협업 리스크 분석 테스트...


2025-06-24 19:47:24,916 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


   결과: 3개 협업 리스크 식별

🔍 개인 리스크 패턴 분석 테스트...
   결과: 0개 개인 리스크 식별

🔍 성과 트렌드 분석 테스트...


2025-06-24 19:47:34,339 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


   결과: 2개 성과 리스크 식별

3️⃣ 성능 비교 테스트 (기존 vs 개선)

🎉 모든 테스트 완료!
